In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
#the tf.keras Sequential model API lets us define our model as a stack of layers

model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

2021-10-08 23:27:48.473817: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

#You can use Keras's handy model.summary() function to see the shape and number of trainable parameters of your model at each layer.

In [12]:
#Ideally you want to see training and validation loss both decreasing
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

2021-10-08 23:30:08.672520: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
209/209 [==============================] - 2s 4ms/step - loss: 8.8337 - mae: 1.5361 - mse: 8.8337 - val_loss: 2.2412 - val_mae: 1.2226 - val_mse: 2.2412
Epoch 2/10
209/209 [==============================] - 1s 3ms/step - loss: 1.5128 - mae: 0.9685 - mse: 1.5128 - val_loss: 1.1708 - val_mae: 0.8364 - val_mse: 1.1708
Epoch 3/10
209/209 [==============================] - 1s 4ms/step - loss: 1.4486 - mae: 0.9457 - mse: 1.4486 - val_loss: 3.2715 - val_mae: 1.5483 - val_mse: 3.2715
Epoch 4/10
209/209 [==============================] - 1s 4ms/step - loss: 1.4162 - mae: 0.9357 - mse: 1.4162 - val_loss: 1.5663 - val_mae: 0.9868 - val_mse: 1.5663
Epoch 5/10
209/209 [==============================] - 1s 3ms/step - loss: 1.3971 - mae: 0.9312 - mse: 1.3971 - val_loss: 1.2440 - val_mae: 0.8842 - val_mse: 1.2440
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3423 - mae: 0.9121 - mse: 1.3423 - val_loss: 1.4235 - val_mae: 0.9525 - val_mse: 1.4235
Epoch 7/10
209/2

In [13]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [14]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  7.8407884
Actual val:  8.24969784404

Predicted val:  8.116399
Actual val:  3.6266042099

Predicted val:  7.9981794
Actual val:  5.74965579296

Predicted val:  8.421995
Actual val:  7.2973008722

Predicted val:  7.947268
Actual val:  7.31273323054

Predicted val:  7.7031302
Actual val:  6.75055446244

Predicted val:  7.932478
Actual val:  6.8122838958

Predicted val:  7.932478
Actual val:  6.75055446244

Predicted val:  7.7681627
Actual val:  6.5918216337999995

Predicted val:  7.511876
Actual val:  5.6879263596



### Using the What-If Tool

In [15]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [16]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [21]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…